# Balancer PID Tuning Utility

### Start Bokeh

In [2]:
from bokeh.plotting import output_notebook, figure, show
from bokeh.models import ColumnDataSource, Range1d
from bokeh.io import push_notebook

output_notebook()

Loading BokehJS ...

In [3]:
from balancer.types import MotionPacket, PIDResponsePacket

source = ColumnDataSource(data=dict(x=[0], y=[0]))
source_pid = ColumnDataSource(data=dict(x=[0], yp=[0], yi=[0], yd=[0]))

p = figure(title="Balancer Pitch", x_axis_label='Sample', y_axis_label='Pitch (Degrees)', width=800, height=480)
line = p.line('x', 'y', source=source, line_width=2, color='blue', alpha=0.8)

p_pid = figure(title="PID Response", x_axis_label='Sample', y_axis_label='Response', width=800, height=480)
p_line = p_pid.line('x', 'yp', source=source_pid, line_width=2, color='red', alpha=0.8)
i_line = p_pid.line('x', 'yi', source=source_pid, line_width=2, color='green', alpha=0.8)
d_line = p_pid.line('x', 'yd', source=source_pid, line_width=2, color='blue', alpha=0.8)

handle = show(p, notebook_handle=True)
handle_pid = show(p_pid, notebook_handle=True)

orientation_counter = -1
def add_orientation_packet(orientation):
    global orientation_counter
    orientation_counter += 1
    new_data = dict(x=[orientation_counter], y=[orientation.pitch])
    source.stream(new_data, rollover=1000)
    push_notebook(handle=handle)

pid_counter = -1
def add_pid_packet(pid):
    global pid_counter
    pid_counter += 1
    new_data = dict(x=[pid_counter], yp=[pid.P], yi=[pid.I], yd=[pid.D])
    source_pid.stream(new_data, rollover=1000)
    push_notebook(handle=handle_pid)

In [4]:
import asyncio
import asyncudp
from balancer.types import MotionPacket, PACKET_TYPE_ORIENTATION, PACKET_TYPE_PID_RESPONSE

stop_listening = asyncio.Event()

async def main():
    sock = await asyncudp.create_socket(local_addr=('0.0.0.0', 3000))
    while not stop_listening.is_set():
        data, addr = await sock.recvfrom()
        packet = MotionPacket.from_buffer_copy(data)
        if packet.type == PACKET_TYPE_ORIENTATION:
            add_orientation_packet(packet.data.orientation)
        elif packet.type == PACKET_TYPE_PID_RESPONSE:
            add_pid_packet(packet.data.pid_response)

def stop():
    stop_listening.set()

In [ ]:
await main()